# Taller 2 - CNN utilizando Keras para el procesamiento de Imágenes

Profesor: Dr. Ing. Rodrigo Salas, Universidad de Valparaíso

[rodrigo.salas@uv.cl](mailto:rodrigo.salas@uv.cl)

Cursillo de Deep Learning aplicado a Imágenes

**Simposio Internacional de Estadística 2025**

**Pasto, Colombia**

## Toolbox Utilizados

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np

## Se utilizará el conjunto de datos de MNIST

Carga del conjunto de datos y separación en entrenamiento y Test

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

Visualización de algunas imágenes como ejemplos

In [ ]:
plt.figure(figsize=(8, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train[i], cmap='gray')
    plt.title(f"Etiqueta: {y_train[i]}")
    plt.axis('off')
plt.suptitle("Ejemplos del conjunto de entrenamiento")

## Preprocesamiento de la imágenes.

Las imágenes son convertidas a números reales con valores en el ranto $[0,1]$

In [ ]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0
X_train = X_train[..., tf.newaxis]  # (n, 28, 28, 1)
X_test = X_test[..., tf.newaxis]

Guardar etiquetas originales para visualización posterior

In [ ]:
y_test_labels = y_test.copy()

Aplicación de *one-hot-encoding* para representar variables categóricas (como letras, clases o etiquetas) en un formato numérico binario donde todas las posiciones son 0, excepto una que es 1, que indica la categoría presente.

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

## Implementación del Modelo de CNN

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

Compilación del modelo utilizando como técnica de aprendizaje de tipo backpropagation el algorimo *ADAM* y la función de pérdida la cross-entropía categórica:

$$L = -\sum_{i=1}^{C} y_i \log(\hat{y}_i)$$

Se visualizará el accuracy durante el entrenamiento.

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary(line_length=80)

Gráfica del Modelo

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='cnn_model.png', show_shapes=True, show_layer_names=True)

## Entrenamiento del modelo de CNN

El modelo de CNN se entrena con 5 épocas, con un batch size de 64, y se utiliza un conjunto de validación con el 20% de los datos.

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    batch_size=64,
                    validation_split=0.2,
                    verbose=1)

Gráfico de la curva de aprendizaje

In [ ]:
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Curva de pérdida')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()

## Evaluación de la Capacidad de Generalización

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Precisión en test: {test_acc:.4f}")

## Ejemplos en el Proceso de Inferencia

In [ ]:
predictions = model.predict(X_test[:10])
pred_labels = np.argmax(predictions, axis=1)

In [ ]:
plt.figure(figsize=(12, 14))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap='gray')
    plt.title(f"Pred: {pred_labels[i]}\nReal: {y_test_labels[i]}")
    plt.axis('off')
plt.suptitle("Predicciones en el conjunto de prueba")
plt.show()

**Gracias**